# Домашнее задание к лекции «Feature Selection»


**Преподаватель:** Юлия Пономарева

**Цель:**

Изучить методы отбора признаков для эффективного обучения моделей машинного обучения.
Описание задания:
В домашнем задании нужно решить задачу классификации точек наиболее эффективно. Для этого в работе необходимо применить различные методы по отбору признаков. Отбор признаков предпочтительнее осуществлять основываясь на математическом аппарате, поэтому данные для этого задания будут сгенерированы, чтобы избежать признаков с физическим смыслом.

**Этапы работы:**

1. Сгенерируйте данные с помощью кода:
from sklearn.datasets import make_classification
x_data_generated, y_data_generated = make_classification(scale=1)

2. Постройте модель логистической регрессии и оцените среднюю точность. Для этого используйте следующий код:
cross_val_score(LogisticRegression(), x, y, scoring=‘accuracy’).mean()

3. Используйте статистические методы для отбора признаков:
* a) Выберите признаки на основе матрицы корреляции.
* b) Отсеките низковариативные признаки (VarianceThreshold).
* c) Повторите п. 2 на отобранных признаках в п. 3a, п. 3b.

4. Осуществите отбор признаков на основе дисперсионного анализа:
* a) Выберите 5 лучших признаков с помощью скоринговой функции для классификации f_classif (SelectKBest(f_classif, k=5)).
* b) Повторите п. 2 на отобранных признаках.

5. Отбор с использованием моделей:
* a) Реализуйте отбор признаков с помощью логистической регрессии. Отобранные признаки подайте далее на вход в саму логистическую регрессию (SelectFromModel). Используйте L1 регуляризацию.
* b) Реализуйте отбор признаков с помощью модели RandomForest и встроенного атрибута feature_impotance.
* c) Повторите п. 2 на отобранных признаках в п. 5a, п. 5b.

6. Перебор признаков:
* a) SequentialFeatureSelector.
* b) Повторите п. 2 на отобранных признаках.

7. Сформулируйте выводы по проделанной работе:
* a) Сделайте таблицу вида |способ выбора признаков|количество признаков|средняя точность модели|.

# Шаг 1: Генерация данных

In [550]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import VarianceThreshold

In [551]:
from sklearn.datasets import make_classification
# Генерация данных с 20 признаками
x_data_generated, y_data_generated = make_classification(scale=1)

# Создание DataFrame со столбцами признаков
df = pd.DataFrame(x_data_generated, columns=[f'feature_{i+1}' for i in range(20)])

# Добавление столбца с целевой переменной
df['target'] = y_data_generated

# Вывод размерности и информации о DataFrame
print("Shape of DataFrame:", df.shape)
print("Dataframe Information:")
print(df.info())

Shape of DataFrame: (100, 21)
Dataframe Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   feature_1   100 non-null    float64
 1   feature_2   100 non-null    float64
 2   feature_3   100 non-null    float64
 3   feature_4   100 non-null    float64
 4   feature_5   100 non-null    float64
 5   feature_6   100 non-null    float64
 6   feature_7   100 non-null    float64
 7   feature_8   100 non-null    float64
 8   feature_9   100 non-null    float64
 9   feature_10  100 non-null    float64
 10  feature_11  100 non-null    float64
 11  feature_12  100 non-null    float64
 12  feature_13  100 non-null    float64
 13  feature_14  100 non-null    float64
 14  feature_15  100 non-null    float64
 15  feature_16  100 non-null    float64
 16  feature_17  100 non-null    float64
 17  feature_18  100 non-null    float64
 18  feature_19  100 no

In [651]:
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,target
0,-0.159718,0.023102,-1.586131,-1.186907,-0.945332,-1.246630,-0.050961,0.313633,0.123222,2.260720,...,0.440385,-0.833534,1.260986,-1.868862,0.871947,-1.131454,0.386170,1.296983,-0.717356,0
1,0.400259,0.934737,-0.049532,-0.452889,-0.358737,1.287906,-0.725186,0.160377,-1.863709,-0.879201,...,-0.995392,-1.620995,-1.535822,0.356780,-0.253251,-1.301427,-0.921258,-0.184597,-0.850401,0
2,0.008808,0.010009,-1.066239,-0.028173,0.716903,0.538708,-0.338293,0.529782,1.630878,0.851296,...,1.558664,-1.021369,-0.042987,-0.700635,-0.602312,-0.753619,0.768150,0.049949,-0.662117,0
3,-0.103581,0.315235,0.502072,1.310006,-1.216598,0.287157,-0.086730,-1.584467,0.022582,1.065719,...,-2.058434,1.106355,0.037855,0.767648,0.212750,1.170956,-1.073916,0.943146,0.718399,1
4,0.038125,1.916712,-0.557175,1.604776,1.275469,1.131179,-1.184532,0.094105,-0.059122,-0.390638,...,2.318047,-1.016850,-1.519583,0.780405,0.637205,-1.542816,-1.422989,0.505314,-0.457500,0


# Шаг 2: Построение модели логистической регрессии

In [682]:
# Готовим целевую и признаки
X = df.drop('target', axis=1)
y = df['target']

In [683]:
# Создание таблицы для сбора метрик разных моделей
results_table = pd.DataFrame(columns=['Model', 'Metric', 'Train', 'Test', 'Num Features'])

In [684]:
# Разбиение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [685]:
# Функция для обучения модели и вычисления метрики accuracy
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    return {
        'Model': model_name,
        'Metric': 'Accuracy',
        'Train': train_accuracy,
        'Test': test_accuracy,
        'Num Features': X_train.shape[1]
    }


In [686]:
# Исходная модель
initial_model = LogisticRegression(max_iter=1000)
initial_metrics = evaluate_model(initial_model, X_train, y_train, X_test, y_test, 'Initial Model')
results_table = pd.concat([results_table, pd.DataFrame([initial_metrics])], ignore_index=True)

In [687]:
results_table

,Model,Metric,Train,Test,Num Features
0,Initial Model,Accuracy,1.0,0.966667,20


1. Train Accuracy (Точность на обучающей выборке): `1.0`
   - Модель предсказывает все обучающие примеры без ошибок (100% точность).
   - Может свидетельствовать о переобучении (overfitting), поскольку слишком высокая точность на обучающей выборке может означать, что модель запомнила примеры из набора данных.

2. Test Accuracy (Точность на тестовой выборке): `0.9666666666666667`
   - Модель предсказывает ~96.67% тестовых примеров правильно.
   - Это хорошая производительность на тестовых данных и говорит о том, что модель достаточно хорошо обобщает на новые данные.

# Шаг 3: Отбор признаков

## а) На основе матрицы корреляции

In [688]:
# Удаление признаков с высокой корреляцией
def remove_highly_correlated_features(data, threshold=0.9):
    corr_matrix = data.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
    return data.drop(to_drop, axis=1)

In [689]:
# Применение фильтрации на тренировочном наборе и обновление тестового набора
X_train_filtered_corr = remove_highly_correlated_features(X_train, threshold=0.9)
X_test_filtered_corr = X_test[X_train_filtered_corr.columns]

In [690]:
# Количество признаков после удаления коррелированных признаков
num_features_after_corr_removal = X_train_filtered_corr.shape[1]

In [691]:
num_features_after_corr_removal

19

## b) На основе низкой вариативности

In [692]:
# Удаление признаков с низкой вариативностью
selector = VarianceThreshold(threshold=0.1)
X_train_filtered_var = selector.fit_transform(X_train_filtered_corr)
X_test_filtered_var = selector.transform(X_test_filtered_corr)

In [693]:
# Количество признаков после удаления признаков с низкой вариативностью
num_features_after_var_removal = X_train_filtered_var.shape[1]

In [694]:
num_features_after_var_removal

19

In [695]:
# Обучение модели после удаления признаков и сравнение метрик
filtered_model = LogisticRegression(max_iter=1000)
filtered_metrics = evaluate_model(filtered_model, X_train_filtered_var, y_train, X_test_filtered_var, y_test, 'Filtered Model')
results_table = pd.concat([results_table, pd.DataFrame([filtered_metrics])], ignore_index=True)

In [696]:
results_table

,Model,Metric,Train,Test,Num Features
0,Initial Model,Accuracy,1.0,0.966667,20
1,Filtered Model,Accuracy,1.0,0.933333,19


1. Переобучение:
   - Обе модели демонстрируют признаки переобучения, так как точность на обучающей выборке равна 100%. Это может указывать на то, что модели слишком хорошо запомнили обучающие данные и могут плохо справляться с новыми данными, которые отличаются от обучающей выборки.

2. Снижение количества признаков:
   - Второй модель была обучена с использованием одного признака меньше, что может свидетельствовать о попытке уменьшить сложность модели и сделать её более обобщающей.

3. Снижение точности на тестовой выборке:
   - Незначительное снижение точности на тестовой выборке (с 96.67% до 93.33%) может говорить о том, что удаленный признак все-таки был важен для хорошей производительности модели, хотя разница в точности невелика.

# Шаг 4: Выбор 5 лучших признаков

Используем библиотеку `sklearn` для выбора 5 лучших признаков с помощью скоринговой функции `f_classif`.

In [697]:
from sklearn.feature_selection import SelectKBest, f_classif

# Выбор 5 лучших признаков
selector = SelectKBest(f_classif, k=5)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Получение выбранных признаков
selected_features = selector.get_support(indices=True)

In [698]:
selected_features

array([ 6, 12, 13, 16, 19])

In [699]:
# Модель, обученная на выбранных признаках
selected_model = LogisticRegression(max_iter=1000)
selected_metrics = evaluate_model(selected_model, X_train_selected, y_train, X_test_selected, y_test, 'Filtered Model (f_classif)')
results_table = pd.concat([results_table, pd.DataFrame([selected_metrics])], ignore_index=True)

In [700]:
results_table

,Model,Metric,Train,Test,Num Features
0,Initial Model,Accuracy,1.0,0.966667,20
1,Filtered Model,Accuracy,1.0,0.933333,19
2,Filtered Model (f_classif),Accuracy,1.0,0.933333,5


Модель с отбором признаков (`Filtered Model (f_classif)`) показывает, что можно значительно уменьшить количество признаков (с 20 до 5), и при этом сохранить высокую точность на тестовой выборке.

 Если точность на тестовой выборке является критически важной, можно оставить начальную модель, так как она показывает наилучшие результаты на тестовой выборке.

# 5. Отбор с использованием моделей:

## Шаг 1: Отбор признаков с помощью Логистической Регрессии (L1 регуляризация)

In [701]:
from sklearn.feature_selection import SelectFromModel

In [702]:
# Обучение модели логистической регрессии с L1 регуляризацией
logistic = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, random_state=42)
logistic.fit(X_train.values, y_train)

# Отбор признаков
selector = SelectFromModel(logistic, prefit=True)

# Получаем список отобранных признаков
feature_idx = selector.get_support()
selected_features = X_train.columns[feature_idx]

# Применение трансформации к данным
X_train_l1 = selector.transform(X_train.values)
X_test_l1 = selector.transform(X_test.values)

# Преобразование в DataFrame с именами признаков
X_train_l1 = pd.DataFrame(X_train_l1, columns=selected_features)
X_test_l1 = pd.DataFrame(X_test_l1, columns=selected_features)

# Модель на отобранных признаках
logistic_selected = LogisticRegression(max_iter=1000, random_state=42)
metrics_l1 = evaluate_model(logistic_selected, X_train_l1, y_train, X_test_l1, y_test, 'Logistic Regression (L1)')
results_table = pd.concat([results_table, pd.DataFrame([metrics_l1])], ignore_index=True)

In [703]:
results_table

,Model,Metric,Train,Test,Num Features
0,Initial Model,Accuracy,1.0,0.966667,20
1,Filtered Model,Accuracy,1.0,0.933333,19
2,Filtered Model (f_classif),Accuracy,1.0,0.933333,5
3,Logistic Regression (L1),Accuracy,1.0,0.966667,9


## Шаг 2: Отбор признаков с помощью RandomForest

In [704]:
from sklearn.ensemble import RandomForestClassifier

In [705]:
# Модель RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Важность признаков
importances = rf.feature_importances_
indices = np.argsort(importances)[-5:]  # Выбираем 5 самых важных признаков

X_train_rf = X_train.iloc[:, indices]
X_test_rf = X_test.iloc[:, indices]

# Модель на отобранных признаках RandomForest
logistic_rf = LogisticRegression(max_iter=1000, random_state=42)
metrics_rf = evaluate_model(logistic_rf, X_train_rf, y_train, X_test_rf, y_test, 'RandomForest Selected Features')
results_table = pd.concat([results_table, pd.DataFrame([metrics_rf])], ignore_index=True)

In [706]:
results_table

,Model,Metric,Train,Test,Num Features
0,Initial Model,Accuracy,1.0,0.966667,20
1,Filtered Model,Accuracy,1.0,0.933333,19
2,Filtered Model (f_classif),Accuracy,1.0,0.933333,5
3,Logistic Regression (L1),Accuracy,1.0,0.966667,9
4,RandomForest Selected Features,Accuracy,1.0,0.966667,5


# 6. Перебор признаков:

SequentialFeatureSelector для пошагового отбора признаков с использованием библиотеки `mlxtend`:

In [707]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Sequential Forward Selection
sfs_forward = SFS(LogisticRegression(max_iter=1000),
                  k_features=5,
                  forward=True,
                  floating=False,
                  scoring='accuracy',
                  cv=5)

# Применение SFS к тренировочным данным
sfs_forward = sfs_forward.fit(X_train, y_train)

# Получение выбранных признаков
selected_features_forward = list(sfs_forward.k_feature_idx_)

# Трансформация данных с использованием выбранных признаков
X_train_sfs_forward = sfs_forward.transform(X_train)
X_test_sfs_forward = sfs_forward.transform(X_test)

# Обучение модели на выбранных признаках и оценка метрик
sfs_forward_model = LogisticRegression(max_iter=1000)
sfs_forward_metrics = evaluate_model(sfs_forward_model, X_train_sfs_forward, y_train, X_test_sfs_forward, y_test, 'SFS Forward Model')
results_table = pd.concat([results_table, pd.DataFrame([sfs_forward_metrics])], ignore_index=True)

In [708]:
results_table

,Model,Metric,Train,Test,Num Features
0,Initial Model,Accuracy,1.000000,0.966667,20
1,Filtered Model,Accuracy,1.000000,0.933333,19
2,Filtered Model (f_classif),Accuracy,1.000000,0.933333,5
3,Logistic Regression (L1),Accuracy,1.000000,0.966667,9
4,RandomForest Selected Features,Accuracy,1.000000,0.966667,5
5,SFS Forward Model,Accuracy,0.985714,0.933333,5


SequentialFeatureSelector для пошагового исключения признаков:

In [709]:
# Sequential Backward Selection
sfs_backward = SFS(LogisticRegression(max_iter=1000),
                   k_features=5,
                   forward=False,
                   floating=False,
                   scoring='accuracy',
                   cv=5)

# Применение SFS к тренировочным данным
sfs_backward = sfs_backward.fit(X_train, y_train)

# Получение выбранных признаков
selected_features_backward = list(sfs_backward.k_feature_idx_)

# Трансформация данных с использованием выбранных признаков
X_train_sfs_backward = sfs_backward.transform(X_train)
X_test_sfs_backward = sfs_backward.transform(X_test)

# Обучение модели на выбранных признаках и оценка метрик
sfs_backward_model = LogisticRegression(max_iter=1000)
sfs_backward_metrics = evaluate_model(sfs_backward_model, X_train_sfs_backward, y_train, X_test_sfs_backward, y_test, 'SFS Backward Model')
results_table = pd.concat([results_table, pd.DataFrame([sfs_backward_metrics])], ignore_index=True)

In [710]:
results_table

,Model,Metric,Train,Test,Num Features
0,Initial Model,Accuracy,1.000000,0.966667,20
1,Filtered Model,Accuracy,1.000000,0.933333,19
2,Filtered Model (f_classif),Accuracy,1.000000,0.933333,5
3,Logistic Regression (L1),Accuracy,1.000000,0.966667,9
4,RandomForest Selected Features,Accuracy,1.000000,0.966667,5
5,SFS Forward Model,Accuracy,0.985714,0.933333,5
6,SFS Backward Model,Accuracy,1.000000,0.966667,5


Начальная модель показывает высокий результат на тренировочном и тестовом наборах данных, используя все 20 признаков. Этот результат содержит вероятность переобучения, учитывая высокую точность на тренировочном наборе.

**Filtered Model:**
Промодель показывает схожий высокий результат при снижении количества признаков до 19. Точность на тестовом наборе немного снизилась, но все еще остается довольно высокой.

**Filtered Model (f_classif):**
Использование статистического метода f_classif для отбора признаков привело к значительному сокращению числа признаков до 5, при этом точность остается высокой на обоих наборах.

**Logistic Regression (L1):**
Регуляризация L1 (Lasso) с логистической регрессией эффективно уменьшила количество признаков до 9, сохраняя высокую точность на тестовом наборе. Это говорит о том, что данный метод хорошо справляется с задачей отбора признаков.

**RandomForest Selected Features:**
Отбор признаков с помощью случайного леса также привел к сокращению признаков до 5 и сохранил высокую точность. Это подтверждает, что метод случайного леса эффективно выделяет важные признаки


**SFS Forward Model:**
Метод пошагового подбора признаков по направлению вперёд (SFS Forward) снизил количество признаков до 5. Точность немного ниже по сравнению с другими методами, но всё ещё остаётся довольно высокой


**SFS Backward Model:**
Метод пошагового подбора признаков по направлению назад (SFS Backward) также уменьшил количество признаков до 5 и сохранил высокую точность, аналогичную другим методам отбора признак


### Общие выводы:
1. Все модели показывают высокую точность. Независимо от метода отбора признаков, модели демонстрируют высокую точность на тренировочном и тестовом наборах данных.
2. Эффективность уменьшения признаков. Методы отбора, такие как f_classif, L1 регуляризация и случайный лес, значительно уменьшают количество признаков до 5-9, почти не теряя точности. Это говорит об их эффективности в выделении наиболее значимых признаков.
3. Проблема переобучения. Несколько моделей показывают точность на тренировочном наборе данных 1.0, что может указывать на переобучение.

Наилучшими методами являются Logistic Regression (L1), RandomForest Selected Features и SFS Backward Model, так как они сократили количество признаков до минимума (5-9 признаков) при сохранении высокой точности на тестовом наборе данных (0.9667).